# REST API
- Representational State Transfer
- Client와 Server가 통신하기 위한 URL 구조에 대한 정의 및 디자인

## 1\. Kakao API

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import requests, json

### 1-1. request token 얻기
- `https://developers.kakao.com/`
- 내 애플리케이션 > 애플리케이션 추가하기

In [2]:
REST_API_KEY = '841d5ba127b64c7c02391f4359ad756e'

### 1-2. KoGPT
- 문서 > KoGPT > REST API
- tokens : 공백을 제외한 글자수

#### 1. 다음문장 만들기

In [ ]:
# 1. document : URL

In [3]:
url = 'https://api.kakaobrain.com/v1/inference/kogpt/generation'

In [4]:
# 2. request(URL: headers, params) > response : json(str)

In [11]:
prompt = '원자폭탄을 개발한 사람은'
params = {'prompt': prompt, 'max_tokens': 50, 'n': 2}
headers = {'Authorization': f'KakaoAK {REST_API_KEY}', 'Content-Type': 'application/json'}

In [12]:
response = requests.post(url, json.dumps(params), headers=headers)
response

<Response [200]>

In [13]:
# json.dumps : 문자열 인코딩 -> 한글은 인터넷 상에서 사용이 안됨 -> 한글 > 영문, 특수문자로 인코딩

In [14]:
json.dumps(params)

'{"prompt": "\\uc6d0\\uc790\\ud3ed\\ud0c4\\uc744 \\uac1c\\ubc1c\\ud55c \\uc0ac\\ub78c\\uc740", "max_tokens": 50, "n": 2}'

In [15]:
response.text

'{"id":"86e62bf9-6422-428c-9b16-eab8c9fbe10e","generations":[{"text":" 미국의 핵과학자 로버트 오펜하이머(오른쪽)와 당시 활약하던 영국 과학자 퍼시 스코트 손턴이다. 최초 수소폭탄은 1945년 7월16일 뉴멕시코 주 앨라모고도 남쪽 30마일 지점에서 실험됐다. /","tokens":50},{"text":" 누구일까? 화학반응으로 원자를 조작하는 방법을 탐색해온 과학자들에 의해서 수수께끼로만 남아 있던 이 문제가 마침내 풀렸다. 1960년대 미국 버클리대학의 해군연구소, 이 연구소에서 물질조작연구실장이","tokens":50}],"usage":{"prompt_tokens":14,"generated_tokens":100,"total_tokens":114}}\n'

In [16]:
# 3. json(str) > parsing : text

In [17]:
texts = response.json()['generations']
texts

[{'text': ' 미국의 핵과학자 로버트 오펜하이머(오른쪽)와 당시 활약하던 영국 과학자 퍼시 스코트 손턴이다. 최초 수소폭탄은 1945년 7월16일 뉴멕시코 주 앨라모고도 남쪽 30마일 지점에서 실험됐다. /',
  'tokens': 50},
 {'text': ' 누구일까? 화학반응으로 원자를 조작하는 방법을 탐색해온 과학자들에 의해서 수수께끼로만 남아 있던 이 문제가 마침내 풀렸다. 1960년대 미국 버클리대학의 해군연구소, 이 연구소에서 물질조작연구실장이',
  'tokens': 50}]

In [18]:
texts = [text['text'] for text in texts]
texts

[' 미국의 핵과학자 로버트 오펜하이머(오른쪽)와 당시 활약하던 영국 과학자 퍼시 스코트 손턴이다. 최초 수소폭탄은 1945년 7월16일 뉴멕시코 주 앨라모고도 남쪽 30마일 지점에서 실험됐다. /',
 ' 누구일까? 화학반응으로 원자를 조작하는 방법을 탐색해온 과학자들에 의해서 수수께끼로만 남아 있던 이 문제가 마침내 풀렸다. 1960년대 미국 버클리대학의 해군연구소, 이 연구소에서 물질조작연구실장이']

#### 2. 문장 분류하기
- 문장 마다 `=긍정`, `=부정` 문자열 작성
- 분류할 마지막 문장 `=` 문자열 작성

In [22]:
# 함수 만들기
def kogpt_api(prompt, command='', max_tokens=128, temperature=1, n=1):
    headers = {'Authorization': f'KakaoAK {REST_API_KEY}', 'Content-Type': 'application/json'}
    params = {'prompt': prompt + command, 'max_tokens': max_tokens, 'temperature': temperature,'n': n}
    response = requests.post(url, json.dumps(params), headers=headers)
    results = response.json()
    results = results['generations']
    return [result['text'] for result in results]

In [23]:
prompt='''상품 후기를 긍정 또는 부정으로 분류합니다.
가격대비좀 부족한게많은듯=부정
재구매 친구들이 좋은 향 난다고 해요=긍정
ㅠㅠ약간 후회가 됩니다..=부정
이전에 먹고 만족해서 재구매합니다=긍정
튼튼하고 잘 쓸수 있을 것 같습니다. 이 가격에 이 퀄리티면 훌륭하죠='''

In [24]:
results = kogpt_api(prompt, max_tokens=1, temperature=0.4)
results

['긍정']

#### 3. 뉴스 한 줄 요약하기
- 마지막에 `한줄 요약:` 문자열 작성

In [28]:
prompt='''미국의 생성형 AI(인공지능) 모델·서비스 개발사 오픈AI(OpenAI)를 상대로 제기된 소송이 누적되고 있다. 쟁점 또한 AI 자료 학습부터 오픈소스
5일 IT(정보기술) 업계와 블룸버그 등 외신에 따르면 일론 머스크 테슬라 CEO(최고경영자)는 미국 캘리포니아주 샌프란시스코에서 오픈AI 법인과
머스크는 소장에서 오픈AI가 마이크로소프트(MS)와 체결한 수십억달러 규모의 파트너십과 사실상 비공개된 AI 기술정보 등에 비춰 오픈AI가 인류의
오픈AI를 둘러싼 법적 분쟁은 지난해 7월 미국 코미디언 겸 작가 사라 실버맨 등이 소송을 제기하면서 부각됐다. 언어모델 훈련 과정에서 창작물이
다만 법원은 챗GPT로 생성된 결과물과 작가들의 원작물 사이의 유사성이 떨어진다는 이유로 지난달 실버맨 등의 청구를 상당 부분 기각한 채 심리를
지난해 12월에는 미국 신문사 뉴욕타임스가 뉴스 저작권 침해를 주장하며 오픈AI와 오픈AI의 주요 투자자 MS를 상대로 소송을 제기했다. 뉴욕타임
업계에선 오픈AI가 지난달 15일 동영상 생성 AI 모델 '소라(Sora)'를 출시한 데 따라 법적 분쟁이 더 복잡해질 것이라는 전망도 나온다. AI 학
법무법인 화우 정보보호센터는 지난달 29일 "데이터를 입력한 주체는 권리침해 등에 대해 책임을 질 가능성이 존재하므로 사전에 입력 데이터에 대한
한줄 요약:'''

results = kogpt_api(prompt, max_tokens=50, temperature=0.4)
results

[" 인공지능(AI), 딥러닝, 머신 러닝, 기계학습...... 4차 산업혁명 시대를 이끌어갈 핵심 키워드입니다. 이들 용어 중 가장 많이 회자되는 단어인 '머신 러닝'은 인간 두뇌처럼 스스로 판단하고 결정할"]

#### 4. 질문에 답변하기
- 문장 마지막에 `?:` 로 끝남

In [35]:
prompt = '''
몬스타엑스의 멤버에는 셔누, 기현, 민혁, 형원, 주헌, 아이엠이 있다.
몬스타엑스 멤버의 이름은?:'''
results = kogpt_api(prompt, temperature=0.2)
results

[' 몬베베 (MONBEBE)']

#### 5. 응용하기

## 2\. Naver API
- 통합검색어 트렌드 API
    - https://datalab.naver.com/
    - https://datalab.naver.com/keyword/trendSearch.naver

### 2-1. Request Token 얻기
- `https://developers.naver.com`
- 1. Request Token 얻기 : 애플리케이션등록 -> app_key 획득
- 2. app_key를 이용해서 데이터 가져오기

In [2]:
# 1. App Key
CLIENT_ID, CLIENT_SECRET = 'gcJgMXckLgYycrcIY8Ur', 'x1S38HeCxs'

# 2. document : url(params, headers)
url = 'https://openapi.naver.com/v1/datalab/search'
params = {
    'startDate': '2018-01-01',
    'endDate': '2024-03-01',
    'timeUnit': 'month',
    'keywordGroups': [
      {'groupName': '트위터', 'keywords': ['트위터', '트윗']},
        {'groupName': '페이스북', 'keywords': ['페이스북', '페북']},
        {'groupName': '인스타그램', 'keywords': ['인스타그램', '인스타']},
    ],
}

headers = {
    'X-Naver-Client-Id': CLIENT_ID,
    'X-Naver-Client-Secret': CLIENT_SECRET,
    'Content-Type': 'application/json',
}

# --header "X-Naver-Client-Id: YOUR_CLIENT_ID" \
# --header "X-Naver-Client-Secret: YOUR_CLIENT_SECRET" \
# --header "Content-Type: application/json" \

# 3. request(url, params, headers) > response : json(str)
response = requests.post(url, json.dumps(params), headers=headers)
response





<Response [200]>

### 2-2. 통합검색어 트렌드 API
- 서비스 : `https://datalab.naver.com/keyword/trendSearch.naver`
- 내 애플리케이션 > dss 애플리케이션 > API 설정 > 데이터랩(검색어 트렌드) 추가
- 사용법 : `https://developers.naver.com/docs/serviceapi/datalab/search/search.md#통합-검색어-트렌드`

In [7]:
# 4. json(str) > list, dict > DataFrame
data = response.json()['results']
len(data)

3

In [12]:
dfs = []
for row in data:
    df = pd.DataFrame(row['data'])
    df['title'] = row['title']
    dfs.append(df)
dfs[1]

,period,ratio,title
0,2018-01-01,97.80481,페이스북
1,2018-02-01,89.05814,페이스북
2,2018-03-01,98.84292,페이스북
3,2018-04-01,100.00000,페이스북
4,2018-05-01,92.23588,페이스북
...,...,...,...
70,2023-11-01,9.97093,페이스북
71,2023-12-01,10.12856,페이스북
72,2024-01-01,10.20900,페이스북
73,2024-02-01,9.19334,페이스북


In [13]:
result = pd.concat(dfs, ignore_index=True)
result

,period,ratio,title
0,2018-01-01,14.59502,트위터
1,2018-02-01,13.02380,트위터
2,2018-03-01,12.97953,트위터
3,2018-04-01,12.82991,트위터
4,2018-05-01,13.05391,트위터
...,...,...,...
220,2023-11-01,23.18797,인스타그램
221,2023-12-01,21.84237,인스타그램
222,2024-01-01,21.69092,인스타그램
223,2024-02-01,19.91368,인스타그램
